In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[langchain] openai
! pip3 install --upgrade --quiet langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00


In [4]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "gen-lang-client-0539303742"

LOCATION = "us-central1"

In [3]:
BUCKET_NAME = "ml_Llama"

BUCKET_URI = f"gs://{BUCKET_NAME}"

In [4]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [5]:
pip install langchain_openai

In [6]:
# Chat completions API
import openai
from google.auth import default, transport
from langchain import PromptTemplate
# Build
from langchain_openai import ChatOpenAI
from vertexai.preview import rag

In [7]:
def reAuth():
  credentials, _ = default()
  auth_request = transport.requests.Request()
  credentials.refresh(auth_request)

  MODEL_LOCATION = "us-central1"

  client = openai.OpenAI(
      base_url=f"https://{MODEL_LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{MODEL_LOCATION}/endpoints/openapi/chat/completions?",
      api_key=credentials.token,
  )
  return client

In [8]:
client = reAuth()

In [9]:
MODEL_ID = "meta/llama-3.1-405b-instruct-maas"

In [10]:
temperature = 0.7
max_tokens = 8192
top_p = 0.85
stream = False

In [11]:
pip install json-repair

In [13]:
import pandas as pd
import json
from json_repair import repair_json  # Ensure this library is installed
import time

# Load the .tsv file
file_path = "/content/drive/MyDrive/Final project/checkTest.tsv"  # Replace with your file path
data = pd.read_csv(file_path, sep='\t')

# Ensure required columns are present
if 'clean_title' not in data.columns or '6_way_label' not in data.columns:
    raise ValueError("The input file must contain 'clean_title' and '6_way_label' columns.")

# Configuration
batch_size = 100 # Adjust the batch size as needed
start_batch = 12 # Specify the batch number to start from
OUTPUT_FILE = "/content/drive/MyDrive/Final project/Llama_modified_data_max.json"
FAILED_FILE = "/content/drive/MyDrive/Final project/Llama_failed_batches_max.json"
SAVE_INTERVAL = 5  # Save progress every 10 batches
flag = 0  # Reauthentication flag

# Placeholder for system instruction
sysInstr = """I will provide article titles in JSON format, each structured across multiple fields. Your task is to subtly modify the `clean_title` to retain its original intent and meaning while making the titles more challenging for fake news detection models, including BERT, CNN, Bi-LSTM, and SVM.

**Modification Guidelines:**
- **Synonym Variation:** Replace words with less common synonyms or alternative expressions to alter surface-level patterns while preserving meaning.
- **Structural Changes:** Rephrase the title by altering the sentence structure (e.g., switching clauses or reordering phrases).
- **Contextual Nuance:** Introduce subtle hints of sarcasm, irony, or ambiguity to challenge the model's ability to infer context accurately.
- **Colloquial Language:** Use idiomatic expressions, informal phrases, or region-specific terms to increase linguistic variability.
- **Negation and Sentiment Shifts:** Add subtle negations, expressions of doubt, or shifts in sentiment to complicate sentiment analysis.
- **Noise Injection:** Incorporate harmless but unusual punctuation, slight spelling variations, or uncommon phrasing to disrupt token patterns.

**Constraints:**
- Do not alter the category or core meaning of the title.
- The title must remain natural, plausible, and coherent, as it would appear in a reliable source.
- The modified titles should challenge model detection by leveraging long-range dependencies, local context changes, and subtle semantic shifts.
- Ensure the response is in valid JSON format, where each row includes `clean_title` and `label`.
- Avoid providing introductions, explanations, or extraneous text. Do not add unnecessary start or stop characters.

Your modifications should aim to evade detection by exploiting weaknesses in the models' ability to process context, handle nuanced language, and recognize long-range dependencies."""
try:
    with open(OUTPUT_FILE, "r") as json_file:
        modified_data = json.load(json_file)
except FileNotFoundError:
    modified_data = []

try:
    with open(FAILED_FILE, "r") as failed_file:
        failed_batches = json.load(failed_file)
except FileNotFoundError:
    failed_batches = []

# Batch processing loop
for iter, i in enumerate(range(start_batch * batch_size, len(data), batch_size), start=start_batch):
    batch_df = data.iloc[i : i + batch_size]  # Extract batch
    print(f"Processing batch {iter + 1}/{(len(data) + batch_size - 1) // batch_size} (Rows {i} to {min(i + batch_size - 1, len(data) - 1)})...")

    # Convert the batch DataFrame to JSON
    batch_json = batch_df.to_json(orient="records")

    # Create prompt for the batch
    prompt = f"""
    Modify the content of the clean_title while keeping the 6_way_label unchanged.
    Articles (JSON):
    {batch_json}
    """
    prompt = sysInstr + "\n" + prompt

    # Reauthenticate the client every 5 batches
    if flag == 3:
        client = reAuth()
        flag = 0

    try:
        # Generate content using the pre-initialized client
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
        )
        modified_text = response.choices[0].message.content

        # Parse the modified JSON response
        json_objects = json.loads(modified_text)

        # Process each object to sanitize the output
        sanitized_objects = []
        for original, modified in zip(batch_df.to_dict(orient="records"), json_objects):
            sanitized_object = {
                "clean_title": modified.get("clean_title", original["clean_title"]).replace("_", " "),  # Replace underscores
                "6_way_label": original["6_way_label"]  # Preserve the original label
            }
            sanitized_objects.append(sanitized_object)

        modified_data.extend(sanitized_objects)  # Add sanitized objects to the list
        print(f"Batch {iter + 1} processed successfully!")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in batch {iter + 1}: {e}")

        # Attempt to repair the JSON if it's malformed
        repaired_text = repair_json(modified_text)
        if repaired_text:
            try:
                json_objects = json.loads(repaired_text)

                # Process repaired JSON for sanitization
                sanitized_objects = []
                if isinstance(json_objects, list):  # Validate it's a list of dictionaries
                    for original, modified in zip(batch_df.to_dict(orient="records"), json_objects):
                        sanitized_object = {
                            "clean_title": modified.get("clean_title", original["clean_title"]).replace("_", " "),  # Replace underscores
                            "6_way_label": original["6_way_label"]
                        }
                        sanitized_objects.append(sanitized_object)
                    modified_data.extend(sanitized_objects)
                    print(f"Successfully repaired JSON for batch {iter + 1}.")
                else:
                    raise ValueError("Repaired JSON is not a list of dictionaries.")
            except Exception as validation_error:
                print(f"Validation failed for repaired JSON in batch {iter + 1}: {validation_error}")
                failed_batches.append({"batch": iter + 1, "response": repaired_text, "prompt": prompt})
        else:
            failed_batches.append({"batch": iter + 1, "response": modified_text, "prompt": prompt})
    except Exception as e:
        print(f"Unexpected error in batch {iter + 1}: {e}")
        failed_batches.append({"batch": iter + 1, "response": modified_text if 'modified_text' in locals() else None, "prompt": prompt})

    flag += 1

    # Save progress periodically
    if iter % SAVE_INTERVAL == 0:  # Save every 10 batches
        with open(OUTPUT_FILE, "w") as json_file:
            json.dump(modified_data, json_file, indent=4)
        with open(FAILED_FILE, "w") as failed_file:
            json.dump(failed_batches, failed_file, indent=4)
        print(f"Intermediate data saved to {OUTPUT_FILE} and {FAILED_FILE}")

# Save the final modified data and failed batches
with open(OUTPUT_FILE, "w") as json_file:
    json.dump(modified_data, json_file, indent=4)
print(f"Modified data saved to {OUTPUT_FILE}")

with open(FAILED_FILE, "w") as failed_file:
    json.dump(failed_batches, failed_file, indent=4)
print(f"Failed batches saved to {FAILED_FILE}")

Processing batch 13/70 (Rows 1200 to 1299)...
Batch 13 processed successfully!
Processing batch 14/70 (Rows 1300 to 1399)...
Error decoding JSON in batch 14: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 14.
Processing batch 15/70 (Rows 1400 to 1499)...
Error decoding JSON in batch 15: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 15.
Processing batch 16/70 (Rows 1500 to 1599)...
Error decoding JSON in batch 16: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 16.
Intermediate data saved to /content/drive/MyDrive/Final project/Llama_modified_data_max.json and /content/drive/MyDrive/Final project/Llama_failed_batches_max.json
Processing batch 17/70 (Rows 1600 to 1699)...
Error decoding JSON in batch 17: Expecting value: line 1 column 1 (char 0)
Successfully repaired JSON for batch 17.
Processing batch 18/70 (Rows 1700 to 1799)...
Error decoding JSON in batch 18: Expecting value: line 1 column